In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyt
import os
import cv2

In [18]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\AEONTP_5026788_checkin_2020-1-13-16-1...,160,349,234,348
1,./images\AEONTP_51F86947_checkin_2020-1-13-16-...,141,399,247,456
2,./images\AEONTP_56P6961_checkin_2020-1-13-16-1...,163,333,247,358
3,./images\AEONTP_62A02636_checkin_2020-1-13-16-...,125,378,196,363
4,./images\AEONTP_6A00530_checkin_2020-1-13-16-1...,89,410,357,459


In [19]:
import xml.etree.ElementTree as xet

In [20]:
def getFilePath(filename):
    image_file_name = xet.parse(filename).getroot().find('filename').text
    image_file_path = os.path.join('./images', image_file_name)
    return image_file_path

In [21]:
image_paths = list(df['filepath'].apply(getFilePath))
image_paths

['./images\\AEONTP_5026788_checkin_2020-1-13-16-19_f7zXETONv.jpg',
 './images\\AEONTP_51F86947_checkin_2020-1-13-16-15sUVxP1Ihlt.jpg',
 './images\\AEONTP_56P6961_checkin_2020-1-13-16-18chzYQvblhX.jpg',
 './images\\AEONTP_62A02636_checkin_2020-1-13-16-14gXI_gF3J5Z.jpg',
 './images\\AEONTP_6A00530_checkin_2020-1-13-16-16i7h_Z3Ua0Y.jpg',
 './images\\AEONTP_6S81U5_checkin_2020-1-13-16-18bx9UOV6rY5.jpg',
 './images\\AQUA1_17054_checkin_2020-10-23-10-17jwhBJ7kzC_.jpg',
 './images\\AQUA1_25697_checkin_2020-10-23-8-17cJpC9WhKCt.jpg',
 './images\\AQUA1_25697_checkin_2020-10-23-8-53U3VaS26nFf.jpg',
 './images\\AQUA1_33928_checkin_2020-10-23-8-46Pi5S8mEXah.jpg',
 './images\\AQUA1_33928_checkin_2020-10-23-8-51ICUxEIQDLw.jpg',
 './images\\AQUA1_40304_checkin_2020-10-25-12-277bdC7U4qUk.jpg',
 './images\\AQUA1_43502_checkin_2020-10-23-8-17ifoUU8ftDr.jpg',
 './images\\AQUA1_43502_checkin_2020-10-23-8-52NVGEHEXUkc.jpg',
 './images\\AQUA1_46294_checkin_2020-10-23-10-18fmUaDsXxvM.jpg',
 './images\\AQUA1_

In [6]:
#### Verify image and output
#### 89	410	357	459

In [22]:
image_path = image_paths[4]
img = cv2.imread(image_path)
cv2.rectangle(img, (89,357), (410, 459), (0, 255, 0), 3)
cv2.namedWindow('example', cv2.WINDOW_NORMAL)
cv2.imshow('example', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [24]:
labels = df.iloc[:, 1:].values

In [25]:
data = []
output = []
for index in range(len(image_paths)):
    image_path = image_paths[index]
    image_arr = cv2.imread(image_path)
    h,w,d = image_arr.shape
    # preprcocessing
    load_image = load_img(image_path, target_size=(224, 224))
    load_image_arr = img_to_array(load_image)
    nor_load_image_arr = load_image_arr/255.0
    # normalize to label
    xmin, xmax, ymin, ymax = labels[index]
    nxmin, nxmax = xmin/w, xmax/w
    nymin, nymax = ymin/h, ymax/h
    label_norm = (nxmin, nxmax, nymin, nymax)
    
    data.append(nor_load_image_arr)
    output.append(label_norm)

In [26]:
X = np.array(data, dtype=np.float32)
Y = np.array(output, dtype=np.float32)
X.shape, Y.shape

((84, 224, 224, 3), (84, 4))

In [27]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.9, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((75, 224, 224, 3), (9, 224, 224, 3), (75, 4), (9, 4))

In [28]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf


In [29]:
inception_resnet = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
inception_resnet.trainable = False
#-------------------------------------
head_model = inception_resnet.output
head_model = Flatten()(head_model)
head_model = Dense(500, activation='relu')(head_model)
head_model = Dense(250, activation='relu')(head_model)
head_model = Dense(4, activation='sigmoid')(head_model)

model = Model(inception_resnet.input, outputs=head_model)

In [30]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

 conv2d_11 (Conv2D)          (None, 25, 25, 64)           12288     ['average_pooling2d[0][0]']   
                                                                                                  
 batch_normalization_5 (Bat  (None, 25, 25, 96)           288       ['conv2d_5[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 batch_normalization_7 (Bat  (None, 25, 25, 64)           192       ['conv2d_7[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 batch_normalization_10 (Ba  (None, 25, 25, 96)           288       ['conv2d_10[0][0]']           
 tchNormalization)                                                                                
          

 block35_1_conv (Conv2D)     (None, 25, 25, 320)          41280     ['block35_1_mixed[0][0]']     
                                                                                                  
 custom_scale_layer (Custom  (None, 25, 25, 320)          0         ['mixed_5b[0][0]',            
 ScaleLayer)                                                         'block35_1_conv[0][0]']      
                                                                                                  
 block35_1_ac (Activation)   (None, 25, 25, 320)          0         ['custom_scale_layer[0][0]']  
                                                                                                  
 conv2d_21 (Conv2D)          (None, 25, 25, 32)           10240     ['block35_1_ac[0][0]']        
                                                                                                  
 batch_normalization_21 (Ba  (None, 25, 25, 32)           96        ['conv2d_21[0][0]']           
 tchNormal

                                                                                                  
 activation_25 (Activation)  (None, 25, 25, 32)           0         ['batch_normalization_25[0][0]
                                                                    ']                            
                                                                                                  
 activation_28 (Activation)  (None, 25, 25, 48)           0         ['batch_normalization_28[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_24 (Conv2D)          (None, 25, 25, 32)           10240     ['block35_2_ac[0][0]']        
                                                                                                  
 conv2d_26 (Conv2D)          (None, 25, 25, 32)           9216      ['activation_25[0][0]']       
          

                                                                                                  
 activation_32 (Activation)  (None, 25, 25, 32)           0         ['batch_normalization_32[0][0]
                                                                    ']                            
                                                                                                  
 activation_35 (Activation)  (None, 25, 25, 64)           0         ['batch_normalization_35[0][0]
                                                                    ']                            
                                                                                                  
 block35_4_mixed (Concatena  (None, 25, 25, 128)          0         ['activation_30[0][0]',       
 te)                                                                 'activation_32[0][0]',       
                                                                     'activation_35[0][0]']       
          

...

 e)                                                                  'activation_170[0][0]']      
                                                                                                  
 block8_2_conv (Conv2D)      (None, 5, 5, 2080)           933920    ['block8_2_mixed[0][0]']      
                                                                                                  
 custom_scale_layer_31 (Cus  (None, 5, 5, 2080)           0         ['block8_1_ac[0][0]',         
 tomScaleLayer)                                                      'block8_2_conv[0][0]']       
                                                                                                  
 block8_2_ac (Activation)    (None, 5, 5, 2080)           0         ['custom_scale_layer_31[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_17

 )                                                                  ]']                           
                                                                                                  
 block8_4_mixed (Concatenat  (None, 5, 5, 448)            0         ['activation_175[0][0]',      
 e)                                                                  'activation_178[0][0]']      
                                                                                                  
 block8_4_conv (Conv2D)      (None, 5, 5, 2080)           933920    ['block8_4_mixed[0][0]']      
                                                                                                  
 custom_scale_layer_33 (Cus  (None, 5, 5, 2080)           0         ['block8_3_ac[0][0]',         
 tomScaleLayer)                                                      'block8_4_conv[0][0]']       
                                                                                                  
 block8_4_

 )                                                                  ]']                           
                                                                                                  
 activation_186 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_186[0][0
 )                                                                  ]']                           
                                                                                                  
 block8_6_mixed (Concatenat  (None, 5, 5, 448)            0         ['activation_183[0][0]',      
 e)                                                                  'activation_186[0][0]']      
                                                                                                  
 block8_6_conv (Conv2D)      (None, 5, 5, 2080)           933920    ['block8_6_mixed[0][0]']      
                                                                                                  
 custom_sc

 atchNormalization)                                                                               
                                                                                                  
 activation_191 (Activation  (None, 5, 5, 192)            0         ['batch_normalization_191[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_194 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_194[0][0
 )                                                                  ]']                           
                                                                                                  
 block8_8_mixed (Concatenat  (None, 5, 5, 448)            0         ['activation_191[0][0]',      
 e)                                                                  'activation_194[0][0]']      
          

 atchNormalization)                                                                               
                                                                                                  
 batch_normalization_202 (B  (None, 5, 5, 256)            768       ['conv2d_202[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_199 (Activation  (None, 5, 5, 192)            0         ['batch_normalization_199[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_202 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_202[0][0
 )                                                                  ]']                           
          

In [31]:
from tensorflow.keras.callbacks import TensorBoard

In [32]:
tfb = TensorBoard('object_detection')

In [33]:
history = model.fit(x=x_train, y=y_train, batch_size=10, epochs=100, validation_data=(x_test, y_test), callbacks=[tfb])

Epoch 1/100

8/8 [==============================] - 29s 2s/step - loss: 0.1273 - val_loss: 0.1327
Epoch 2/100
8/8 [==============================] - 9s 1s/step - loss: 0.1273 - val_loss: 0.1202
Epoch 3/100
8/8 [==============================] - 10s 1s/step - loss: 0.1201 - val_loss: 0.1163
Epoch 4/100
8/8 [==============================] - 11s 1s/step - loss: 0.1154 - val_loss: 0.1095
Epoch 5/100
8/8 [==============================] - 11s 1s/step - loss: 0.1110 - val_loss: 0.1176
Epoch 6/100
8/8 [==============================] - 10s 1s/step - loss: 0.1033 - val_loss: 0.1150
Epoch 7/100
8/8 [==============================] - 10s 1s/step - loss: 0.0974 - val_loss: 0.1025
Epoch 8/100
8/8 [==============================] - 11s 1s/step - loss: 0.0913 - val_loss: 0.0981
Epoch 9/100
8/8 [==============================] - 11s 1s/step - loss: 0.0887 - val_loss: 0.1050
Epoch 10/100
8/8 [==============================] - 10s 1s/step - loss: 0.0966 - val_loss: 0.0843
Epoch 11/100
8/8 [===========

8/8 [==============================] - 11s 1s/step - loss: 0.0491 - val_loss: 0.0554
Epoch 83/100
8/8 [==============================] - 13s 2s/step - loss: 0.0487 - val_loss: 0.0532
Epoch 84/100
8/8 [==============================] - 13s 2s/step - loss: 0.0438 - val_loss: 0.0485
Epoch 85/100
8/8 [==============================] - 12s 2s/step - loss: 0.0363 - val_loss: 0.0448
Epoch 86/100
8/8 [==============================] - 13s 2s/step - loss: 0.0343 - val_loss: 0.0394
Epoch 87/100
8/8 [==============================] - 12s 2s/step - loss: 0.0336 - val_loss: 0.0358
Epoch 88/100
8/8 [==============================] - 13s 2s/step - loss: 0.0338 - val_loss: 0.0429
Epoch 89/100
8/8 [==============================] - 12s 1s/step - loss: 0.0338 - val_loss: 0.0400
Epoch 90/100
8/8 [==============================] - 11s 1s/step - loss: 0.0336 - val_loss: 0.0373
Epoch 91/100
8/8 [==============================] - 10s 1s/step - loss: 0.0337 - val_loss: 0.0373
Epoch 92/100
8/8 [===============

In [34]:
history = model.fit(x=x_train, y=y_train, batch_size=10, epochs=200, validation_data=(x_test, y_test), callbacks=[tfb], initial_epoch=100)

Epoch 101/200
8/8 [==============================] - 10s 1s/step - loss: 0.0304 - val_loss: 0.0369
Epoch 102/200
8/8 [==============================] - 9s 1s/step - loss: 0.0300 - val_loss: 0.0359
Epoch 103/200
8/8 [==============================] - 10s 1s/step - loss: 0.0318 - val_loss: 0.0372
Epoch 104/200
8/8 [==============================] - 13s 2s/step - loss: 0.0307 - val_loss: 0.0359
Epoch 105/200
8/8 [==============================] - 13s 2s/step - loss: 0.0304 - val_loss: 0.0360
Epoch 106/200
8/8 [==============================] - 12s 1s/step - loss: 0.0302 - val_loss: 0.0361
Epoch 107/200
8/8 [==============================] - 13s 2s/step - loss: 0.0301 - val_loss: 0.0358
Epoch 108/200
8/8 [==============================] - 19s 2s/step - loss: 0.0299 - val_loss: 0.0347
Epoch 109/200
8/8 [==============================] - 14s 2s/step - loss: 0.0291 - val_loss: 0.0348
Epoch 110/200
8/8 [==============================] - 12s 2s/step - loss: 0.0298 - val_loss: 0.0355
Epoch 111/2

8/8 [==============================] - 17s 2s/step - loss: 6.0882e-04 - val_loss: 0.0081
Epoch 184/200
8/8 [==============================] - 13s 2s/step - loss: 4.7209e-04 - val_loss: 0.0076
Epoch 185/200
8/8 [==============================] - 13s 2s/step - loss: 4.5159e-04 - val_loss: 0.0073
Epoch 186/200
8/8 [==============================] - 15s 2s/step - loss: 4.3666e-04 - val_loss: 0.0076
Epoch 187/200
8/8 [==============================] - 14s 2s/step - loss: 4.3171e-04 - val_loss: 0.0077
Epoch 188/200
8/8 [==============================] - 16s 2s/step - loss: 4.1668e-04 - val_loss: 0.0080
Epoch 189/200
8/8 [==============================] - 13s 2s/step - loss: 5.3231e-04 - val_loss: 0.0084
Epoch 190/200
8/8 [==============================] - 20s 3s/step - loss: 4.6782e-04 - val_loss: 0.0082
Epoch 191/200
8/8 [==============================] - 20s 2s/step - loss: 5.7804e-04 - val_loss: 0.0072
Epoch 192/200
8/8 [==============================] - 15s 2s/step - loss: 4.6947e-04 - v

In [35]:
model.save('./models/object_detection.h5')

C:\Users\Iamkh\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
